### Some example code demonstrating some super() behaviour


In [1]:
class A(object):
    def __init__(self):
        print "in A __init__"
        print "self's class is:", self.__class__
        s = super(A, self).__init__()

class B(object):
    def __init__(self):
        print "in B.__init__"
        s = super(B, self).__init__()

class C(object):
    def __init__(self):
        print "in C.__init__"
        s = super(C, self).__init__()

class D(C, B, A):
    def __init__(self):
        print "self's class is:", self.__class__
        super(D, self).__init__()

In [2]:
print "D's mro:", D.__mro__
d = D()

D's mro: (<class '__main__.D'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <type 'object'>)
self's class is: <class '__main__.D'>
in C.__init__
in B.__init__
in A __init__
self's class is: <class '__main__.D'>


In [3]:
s_c = super(C, d)
print s_c

<super: <class 'C'>, <D object>>


In [4]:
c = C()

in C.__init__


In [5]:
super(D, c)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [6]:
s_a = super(A, d)
print s_a

<super: <class 'A'>, <D object>>


In [7]:
s_b = super(B, d)
print s_b

<super: <class 'B'>, <D object>>


In [8]:
print D.__mro__

(<class '__main__.D'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <type 'object'>)


In [ ]:
## why use super() everywhere?

In [9]:
# the "old way":

class A(object):
    def this(self):
        print "in A.this"

class B(object):
    def this(self):
        print "in B.this"

class C(A,B):
    def this(self):
        print "in C.this"
        A.this(self)
        B.this(self)

In [10]:
print "Running the traditional way"
c = C()
c.this()

Running the traditional way
in C.this
in A.this
in B.this


### Using super in just C:

In [23]:
class A(object):
    def this(self):
        print "in A.this"

class B(A):
    def this(self):
        print "in B.this"

class C(B):
    def this(self):
        print "in C.this"
        super(C, self).this()

In [24]:
print C.__mro__
c = C()
c.this()

(<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <type 'object'>)
in C.this
in B.this


**Note:**  `B.this` did NOT get called!

Even though it is in in the MRO:

In [13]:
C.__mro__

(__main__.C, __main__.A, __main__.B, object)

### Using super everywhere:

In [26]:
class Base(object):
    def this(self):
        pass # just so there is a base that has the method

class A(Base):
    def this(self):
        print "in A.this"
        super(A, self).this()

class B(Base):
    def this(self):
        print "in B.this"
        super(B, self).this()
class C(A,B):
    def this(self):
        print "in C.this"
        super(C, self).this()

In [27]:
c = C()
c.this()
print Base.__mro__

in C.this
in A.this
in B.this
(<class '__main__.Base'>, <type 'object'>)


Now both A and B's methods get called -- probably what you want.

But if you don't want both called -- better to just be Explicit, rather than use super():

In [32]:
class Base(object):
    def this(self):
        pass # just so there is a base that has the method

class A(Base):
    def this(self):
        print "in A.this"
        super(A, self).this()

class B(Base):
    def this(self):
        print "in B.this"
        super(B, self).this()

class C(A,B):
    def this(self):
        print "in C.this"
        A.this(self)

In [30]:
c = C()
c.this()

in C.this
in A.this
in B.this


**Whoa** -- B's method DID get called! -- why?

In [33]:
A.__mro__

(__main__.A, __main__.Base, object)

B is not there.

But:

In [34]:
class Base(object):
    def this(self):
        pass # just so there is a base that has the method

class A(Base):
    def this(self):
        print "in A.this"
        print "self's class:", self.__class__
        super(A, self).this()

class B(Base):
    def this(self):
        print "in B.this"
        super(B, self).this()

class C(A,B):
    def this(self):
        print "in C.this"
        A.this(self)

In [36]:
c = C()
c.this()
print C.__mro__

in C.this
in A.this
self's class: <class '__main__.C'>
in B.this
(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <type 'object'>)


Remember, `super()` is dynamic -- what it calls is determined at run time.

That's why you need to pass `self` in to `super()`, and how it knows to call ``B``'s method too. Which is why we say that using `super()` is *part* of the interface of the class.